In [2]:
addprocs(30);

In [3]:
@everywhere begin
    using POMDPModels
    using POMCP
    using POMDPs
    import POMDPToolbox.PreviousObservation

    import POMCP.belief_from_node
    import POMCP.init_V
    # import POMCP.POMCPPolicy
    import POMDPs.action
end

In [16]:
N = 100000;
problem = BabyPOMDP(-5, -10);

In [17]:
@everywhere type RandomBabyPolicy <: POMDPs.Policy
    rng::AbstractRNG
end
@everywhere action(p::RandomBabyPolicy, b::POMDPs.Belief) = BabyAction(rand(p.rng)>0.5)

In [18]:
function est_reward(problem, policy, belief, N; eps=0.01)
    sum = @parallel (+) for i in 1:N
        sim_rng = MersenneTwister(i)
        POMDPs.simulate(problem, policy, belief, rng=sim_rng, eps=eps, initial_state=BabyState(false))
    end
    return sum/N;
end

est_reward (generic function with 1 method)

In [20]:
# Feed when Crying (Expected Reward for this nearly-optimal policy is -17.14)
# Test from earlier this week with 5000 experiments: -16.72
@time est_reward(problem, FeedWhenCrying(), PreviousObservation(BabyObservation(false)), N)

elapsed time: 0.381954495 seconds (1000404 bytes allocated)


-16.621810988816325

This is better than in the crying babies test because epsilon is large and, more importantly, it gets a notcrying observation on the first step every time

In [21]:
# Random
pol_rng = MersenneTwister(7)
@time est_reward(problem, RandomBabyPolicy(pol_rng), PreviousObservation(BabyObservation(false)), N)

elapsed time: 0.440277441 seconds (1258044 bytes allocated)


-32.17854039299618

In [22]:
# POMCP with FWC rollout policy
# test from earlier this week with 5000 experiments: -16.77
rng = MersenneTwister(2)

solver = POMCPSolver(FeedWhenCrying(),
                     0.01,
                     10,
                     300, 
                     rng,
                     false,
                     PreviousObservationConverter())

policy = solve(solver, problem)

@time est_reward(problem, policy, POMCPBeliefWrapper(BabyStateDistribution(0.0)), N)

elapsed time: 4226.238562773 seconds (1241024 bytes allocated)


-16.263003916559974

In [23]:
# POMCP with Random rollout policy
rng = MersenneTwister(2)
rollout_pol_rng = MersenneTwister(2)

solver = POMCPSolver(RandomBabyPolicy(rollout_pol_rng),
                     0.01,
                     10,
                     300, 
                     rng,
                     false,
                     EmptyConverter())

policy = solve(solver, problem)

@time est_reward(problem, policy, POMCPBeliefWrapper(BabyStateDistribution(0.0)), N)

elapsed time: 2980.964993092 seconds (1671768 bytes allocated)


-17.209711843801013

In [24]:
# Optimal policy for these particular problem parameters:
# if the belief that the baby is hungry is over .28206, then feed (see DMU book)
@everywhere begin
    type OptBabyPolicy <: POMDPs.Policy
    end
    action(p::OptBabyPolicy, b::BabyStateDistribution) = BabyAction(b.p_hungry>0.28206)
end
@time est_reward(problem, OptBabyPolicy(), BabyStateDistribution(0.0), N)

elapsed time: 0.328451778 seconds (1278340 bytes allocated, 13.51% gc time)


-16.16069990155499